In [63]:
import pandas as pd
import sqlite3

In [64]:
import pandas as pd

def processar_vendas_excel(caminho_arquivo, nome_aba="Sheet1"):
    """
    Processa um arquivo Excel contendo dados de vendas.
    - Identifica corretamente a linha do produto e associa às vendas anteriores.
    - Remove linhas de 'Total Produto'.
    - Separa código do produto e descrição corretamente.
    - Converte colunas para tipos adequados.

    Retorna:
    - DataFrame processado com colunas organizadas.
    """

    # Carregar o arquivo Excel
    df_vendas = pd.read_excel(caminho_arquivo, sheet_name=nome_aba)

    # Criar coluna para armazenar os produtos corretamente
    df_vendas["Produto"] = None
    produto_atual = None
    linhas_para_atualizar = []  # Armazena índices das linhas de venda

    for index, row in df_vendas.iterrows():
        descricao = str(row["Núm. Venda"])

        # Se encontramos um "Total Produto", pegamos o nome do produto
        if "Total Produto" in descricao:
            partes = descricao.split("Total Produto:")
            if len(partes) > 1:
                produto_atual = partes[1].strip()  # Nome correto do produto

            # Agora aplicamos esse produto a todas as linhas anteriores que precisam dele
            for i in linhas_para_atualizar:
                df_vendas.at[i, "Produto"] = produto_atual

            # Resetamos a lista, pois um novo produto será associado a novas linhas
            linhas_para_atualizar = []

        else:
            # Armazena o índice da linha que pertence ao produto atual
            linhas_para_atualizar.append(index)

    # Remover linhas de "Total Produto" (que não são vendas)
    df_vendas = df_vendas.dropna(subset=["Produto"])

    # Separar Código do Produto e Descrição do Produto corretamente
    df_vendas[["codigo", "produto"]] = df_vendas["Produto"].str.split(" ", n=1, expand=True)

    # Remover a coluna antiga "Produto"
    df_vendas = df_vendas.drop(columns=["Produto"])

    # Selecionar e renomear colunas relevantes
    df_vendas = df_vendas[[
        "Emissão", "Núm. Venda", "codigo", "produto", "Loja",
        "Qtd. Vend.", "Vlr.Unit", "Vlr.Total"
    ]].copy()

    # Renomear colunas para o formato final
    df_vendas.columns = [
        "data", "num_venda", "codigo", "produto", "loja",
        "qtd_vendida", "preco_unit", "valor_total"
    ]

    # Converter colunas para os formatos corretos
    df_vendas["data"] = pd.to_datetime(df_vendas["data"], errors="coerce").dt.date  # Apenas a data
    df_vendas["qtd_vendida"] = pd.to_numeric(df_vendas["qtd_vendida"], errors="coerce")
    df_vendas["preco_unit"] = pd.to_numeric(df_vendas["preco_unit"].astype(str).str.replace(',', '.'), errors="coerce")
    df_vendas["valor_total"] = pd.to_numeric(df_vendas["valor_total"].astype(str).str.replace(',', '.'), errors="coerce")

    return df_vendas


In [65]:
df = processar_vendas_excel('planilhas/vendas/produtos/vendas_produtos_022025.xlsx')

z:\spx\venv\lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Samsung\AppData\Local\Temp\ipykernel_5760\2044499515.py:65: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_vendas["data"] = pd.to_datetime(df_vendas["data"], errors="coerce").dt.date  # Apenas a data


In [66]:
df.head(60)

,data,num_venda,codigo,produto,loja,qtd_vendida,preco_unit,valor_total
0,2025-03-02,44648,54135,20 BI 335MG 5CAP,9.0,1.0,59.99,59.99
2,2025-06-02,69842,130047,5STAR CHOC CAR 40G,10.0,2.0,7.99,15.98
3,2025-04-02,69170,130047,5STAR CHOC CAR 40G,10.0,1.0,7.99,7.99
5,2025-07-02,13965,2219,AAS 100MG 30CP,11.0,1.0,23.99,23.99
7,2025-05-02,47560,8643,AAS PROTECT 100MG 30CP REV L.E,7.0,1.0,20.99,20.99
9,2025-09-02,47999,150077,AB.SEIOS COTTONBABY MATERNA 30,7.0,1.0,35.99,35.99
11,2025-09-02,162320,8634,ABC 10MG/G CR DERM 20G,1.0,1.0,16.99,16.99
13,2025-05-02,50843,15892,ABC 10MG/ML SPR DERM 30ML,8.0,1.0,33.99,33.99
15,2025-07-02,70389,16273,ABIDOR 200MG/ML GTS 15ML,10.0,1.0,12.05,12.05
16,2025-02-02,68612,16273,ABIDOR 200MG/ML GTS 15ML,10.0,1.0,12.05,12.05


In [67]:
df.query("codigo == '11783'")

,data,num_venda,codigo,produto,loja,qtd_vendida,preco_unit,valor_total
34234,2025-06-02,1176,11783,TORSILAX 30CP,12.0,3.0,16.99,25.49
34235,2025-02-02,123336,11783,TORSILAX 30CP,5.0,1.0,16.99,16.99
34236,2025-07-02,47781,11783,TORSILAX 30CP,7.0,1.0,16.99,16.99
34237,2025-01-02,44089,11783,TORSILAX 30CP,9.0,1.0,16.99,15.19
34238,2025-08-02,46365,11783,TORSILAX 30CP,9.0,1.0,16.99,16.99
...,...,...,...,...,...,...,...,...
34366,2025-03-02,161623,11783,TORSILAX 30CP,1.0,1.0,16.99,16.99
34367,2025-03-02,161621,11783,TORSILAX 30CP,1.0,1.0,16.99,16.99
34368,2025-01-02,47107,11783,TORSILAX 30CP,7.0,1.0,16.99,16.99
34369,2025-02-02,123343,11783,TORSILAX 30CP,5.0,1.0,16.99,16.99


In [68]:
import sqlite3

def salvar_dataframe_no_sqlite(df_vendas, banco_dados="vendas.db"):
    """
    Salva um DataFrame no banco de dados SQLite, evitando duplicação de vendas.

    Cada venda é identificada de forma única por:
    - data
    - num_venda
    - codigo do produto

    Se um registro já existir, ele NÃO será inserido novamente.
    """

    # Conectar ao banco SQLite
    conn = sqlite3.connect(banco_dados)
    cursor = conn.cursor()

    # Criar tabela se não existir
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS vendas (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            data TEXT,
            num_venda INTEGER,
            codigo TEXT,
            produto TEXT,
            loja INTEGER,
            qtd_vendida INTEGER,
            preco_unit REAL,
            valor_total REAL,
            UNIQUE (data, num_venda, codigo, loja, qtd_vendida)  -- Permite vários produtos por venda, mas impede duplicatas
        )
    ''')
    conn.commit()

    # Converter a coluna de data para string
    df_vendas["data"] = df_vendas["data"].astype(str)

    # Contador de registros inseridos
    inseridos = 0

    # Inserir dados manualmente para evitar erro de integridade
    for _, row in df_vendas.iterrows():
        cursor.execute('''
            INSERT OR IGNORE INTO vendas (data, num_venda, codigo, produto, loja, qtd_vendida, preco_unit, valor_total)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (row["data"], row["num_venda"], row["codigo"], row["produto"], row["loja"], row["qtd_vendida"], row["preco_unit"], row["valor_total"]))

        # Se a venda foi inserida, o cursor.rowcount será 1
        if cursor.rowcount > 0:
            inseridos += 1

    conn.commit()
    print(f"{inseridos} novas vendas adicionadas ao banco de dados.")  # Agora conta corretamente

    # Fechar conexão com o banco
    conn.close()



In [69]:
salvar_dataframe_no_sqlite(df)

0 novas vendas adicionadas ao banco de dados.


In [70]:
def consultar_por_produto(produto_parcial, banco_dados="vendas.db"):
    """
    Consulta os dados de vendas filtrando por um nome parcial do produto.
    
    Parâmetros:
    - produto_parcial: Parte do nome do produto que deseja buscar.
    - banco_dados: Nome do banco SQLite.

    Retorna:
    - DataFrame com os resultados filtrados.
    """
    conn = sqlite3.connect(banco_dados)

    # Remover espaços extras e converter para maiúsculas
    produto_parcial = produto_parcial.strip().upper()

    # Consulta SQL para buscar produtos que contenham a palavra digitada
    query = "SELECT * FROM vendas WHERE UPPER(produto) LIKE ?"
    
    # Usamos o '%' antes e depois para encontrar qualquer produto que tenha essa palavra no nome
    df_vendas = pd.read_sql(query, conn, params=(f"%{produto_parcial}%",))

    conn.close()
    
    return df_vendas



In [71]:
# Exemplo de consulta
df_filtrado = consultar_por_produto("torsilax 30")
print(df_filtrado)

          id        data  num_venda codigo         produto  loja  qtd_vendida  \
0      52387  2024-09-01      82647  11783   TORSILAX 30CP     5            1   
1      52388  2024-01-24      84450  11783   TORSILAX 30CP     5            1   
2      52389  2024-01-19       6427  11783   TORSILAX 30CP     9            1   
3      52390  2024-01-19     104074  11783   TORSILAX 30CP     4            1   
4      52391  2024-01-28       1539  11783   TORSILAX 30CP    10            1   
...      ...         ...        ...    ...             ...   ...          ...   
4884  641100  2025-03-02     161623  11783   TORSILAX 30CP     1            1   
4885  641101  2025-03-02     161621  11783   TORSILAX 30CP     1            1   
4886  641102  2025-01-02      47107  11783   TORSILAX 30CP     7            1   
4887  641103  2025-02-02     123343  11783   TORSILAX 30CP     5            1   
4888  641104  2025-02-02      44321  11783   TORSILAX 30CP     9            1   

      preco_unit  valor_tot

In [72]:
import sqlite3
import pandas as pd

def listar_produtos_unicos(banco_dados="vendas.db"):
    """
    Lista todos os produtos únicos armazenados no banco de dados.
    """
    conn = sqlite3.connect(banco_dados)
    query = "SELECT DISTINCT produto FROM vendas ORDER BY produto"
    df_produtos = pd.read_sql(query, conn)
    conn.close()
    return df_produtos

# Listar todos os produtos para verificar os nomes corretos
df_produtos = listar_produtos_unicos()
print(df_produtos)


                              produto
0       *PROV GEL FIXA BRONZE POS SOL
1                   20 BI 335MG 10CAP
2                   20 BI 335MG 30CAP
3                    20 BI 335MG 5CAP
4        212 VIP MEN EDT 100ML ADIPEC
...                               ...
18081         ZYMAR XD 5MG/ML COL 5ML
18082      ZYPRED 3MG+10MG/ML COL 6ML
18083            ZYRTEC 10MG 12CP REV
18084              ZYXEM 5MG 10CP REV
18085           ZYXEM 5MG/ML GTS 20ML

[18086 rows x 1 columns]
